In [22]:
import pandas as pd
import time

In [23]:
start=time.perf_counter()

In [24]:
from DatabaseCreator import DatabaseCreator
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [25]:
#add this info to a settings dict?
stats_current_gameweek = 9
picks_current_gameweek = 12
current_season = 20252026

In [33]:
print("Check status of available players")
view = db_creator.run_sql(f"""
    
    WITH predictions_status AS (
    SELECT *
    FROM predictions
    LEFT JOIN player_status ON predictions.element = player_status.id
    )
    SELECT
        web_name,
        status,
        chance_of_playing_this_round,
        event,
        position,
        team_name,
        opp_team_name,
        xpoints
    FROM predictions_status
    WHERE in_my_team = 0
        AND in_any_team = 0
        AND event = {picks_current_gameweek}
        AND status = 'a'
    ORDER BY xpoints DESC
""")
view.head(5)

Check status of available players


,web_name,status,chance_of_playing_this_round,event,position,team_name,opp_team_name,xpoints
0,Acheampong,a,100.0,12.0,DEF,Chelsea,Burnley,5.141233
1,Tavernier,a,NaN,12.0,MID,Bournemouth,West Ham,4.802883
2,Calvert-Lewin,a,75.0,12.0,FWD,Leeds,Aston Villa,4.607547
3,Wharton,a,100.0,12.0,MID,Crystal Palace,Wolves,4.524026
4,Zubimendi,a,75.0,12.0,MID,Arsenal,Spurs,4.499787


In [27]:
print("When to use first TRIPLE CAPTAIN chip (gws1-19)")
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        event,
        position,
        team_name,
        opp_team_name,
        xpoints
    FROM predictions
    WHERE in_my_team = 1
      AND event BETWEEN {picks_current_gameweek} AND 19
    ORDER BY xpoints DESC
""")
view.head(5)

When to use first TRIPLE CAPTAIN chip (gws1-19)


,player_name_id,event,position,team_name,opp_team_name,xpoints
0,Bruno Borges Fernandes,19.0,MID,Man Utd,Wolves,6.527842
1,Bruno Borges Fernandes,15.0,MID,Man Utd,Wolves,6.527842
2,Gabriel dos Santos MagalhÃ£es,16.0,DEF,Arsenal,Wolves,6.197677
3,Matheus Nunes,13.0,DEF,Man City,Leeds,6.036608
4,Bruno Borges Fernandes,14.0,MID,Man Utd,West Ham,5.979741


In [28]:
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        in_my_team,
        position,
        team_name,
        ROUND(AVG(opp_team_elo),1) AS avg_opp_team_elo,
        ROUND(SUM(xpoints),2) AS total_xpoints,
        ROUND(AVG(mins_pred),1) AS avg_mins_pred,
        ROUND(AVG(xgp90_pred),3) AS avg_xgp90_pred,
        ROUND(AVG(xap90_pred),3) AS avg_xap90_pred,
        ROUND(AVG(xgcp90_pred),3) AS avg_xgcp90_pred
    FROM predictions
    WHERE (in_any_team = 0 OR in_my_team = 1)
      AND event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
      -- AND position = 'FWD'
    GROUP BY player_name_id, position, team_name, in_my_team
    ORDER BY total_xpoints DESC
""")
view.head(20)

,player_name_id,in_my_team,position,team_name,avg_opp_team_elo,total_xpoints,avg_mins_pred,avg_xgp90_pred,avg_xap90_pred,avg_xgcp90_pred
0,Gabriel dos Santos MagalhÃ£es,1,DEF,Arsenal,1056.7,33.00,83.6,0.122,0.072,0.887
1,Bruno Borges Fernandes,1,MID,Man Utd,1044.4,32.99,74.6,0.339,0.241,1.268
2,Matheus Nunes,1,DEF,Man City,1024.1,30.80,79.6,0.052,0.069,0.852
3,Dominik Szoboszlai,1,MID,Liverpool,1011.4,30.30,82.1,0.208,0.251,1.235
4,Dango Ouattara,1,MID,Bournemouth,1028.4,28.73,73.1,0.304,0.123,1.303
5,Bruno GuimarÃ£es Rodriguez Moura,0,MID,Newcastle,1063.3,28.61,78.3,0.226,0.169,1.284
6,Marcus Tavernier,0,MID,Bournemouth,1028.4,27.97,78.9,0.181,0.184,1.216
7,Dominic Calvert-Lewin,0,FWD,Leeds,1146.4,27.96,76.5,0.522,0.062,2.014
8,Yankuba Minteh,1,MID,Brighton,1074.6,27.61,80.3,0.195,0.175,1.462
9,Amad Diallo,1,MID,Man Utd,1044.4,27.58,61.7,0.283,0.136,1.159


In [29]:
print(f"\nBest available players to replace worst in my team over next 5 GWs: {picks_current_gameweek+1} to 38")#{picks_current_gameweek + 5}
view = db_creator.run_sql(f"""
    WITH 
    summed AS (
        SELECT 
            player_name_id,
            team_name,
            position,
            in_my_team,
            in_any_team,
            SUM(xpoints) AS total_xpoints
        FROM predictions
        WHERE event BETWEEN {picks_current_gameweek+1} AND 38 --{picks_current_gameweek + 5}
        GROUP BY player_name_id, team_name, position, in_my_team, in_any_team
    ),

    worst_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'GK'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_gk AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'GK'
    ),

    worst_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'DEF'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_def AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'DEF'
    ),

    worst_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'MID'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_mid AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'MID'
    ),

    worst_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_my_team = 1
          AND position = 'FWD'
        ORDER BY total_xpoints ASC
        LIMIT 1
    ),
    best_fwd AS (
        SELECT player_name_id, team_name, total_xpoints AS xpoints, in_my_team, position
        FROM summed
        WHERE in_any_team = 0
          AND in_my_team = 0
          AND position = 'FWD'
    )

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_gk b
    CROSS JOIN worst_gk w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_def b
    CROSS JOIN worst_def w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_mid b
    CROSS JOIN worst_mid w

    UNION ALL

    SELECT 
        b.position,
        b.team_name AS best_team,
        b.player_name_id AS best_player_id,
        b.xpoints AS best_xpoints,
        w.player_name_id AS worst_player_id,
        w.xpoints AS worst_xpoints,
        (b.xpoints - w.xpoints) AS xpoints_diff
    FROM best_fwd b
    CROSS JOIN worst_fwd w
    ORDER BY xpoints_diff DESC
""")

view.head(20)



Best available players to replace worst in my team over next 5 GWs: 13 to 38


,position,best_team,best_player_id,best_xpoints,worst_player_id,worst_xpoints,xpoints_diff
0,FWD,Leeds,Dominic Calvert-Lewin,124.021723,Chris Wood,56.025571,67.996152
1,FWD,Burnley,Zian Flemming,106.512249,Chris Wood,56.025571,50.486678
2,FWD,Fulham,RaÃºl JimÃ©nez RodrÃ­guez,104.474209,Chris Wood,56.025571,48.448639
3,FWD,Leeds,Lukas Nmecha,74.455452,Chris Wood,56.025571,18.429881
4,FWD,Brighton,Charalampos Kostoulas,74.119387,Chris Wood,56.025571,18.093816
5,FWD,Nott'm Forest,Taiwo Awoniyi,73.541054,Chris Wood,56.025571,17.515484
6,FWD,Chelsea,Marc Guiu Paz,71.394041,Chris Wood,56.025571,15.368470
7,FWD,Crystal Palace,Eddie Nketiah,69.908428,Chris Wood,56.025571,13.882858
8,FWD,Everton,Norberto Bercique Gomes Betuncal,69.371437,Chris Wood,56.025571,13.345867
9,FWD,Brighton,Stefanos Tzimas,66.241733,Chris Wood,56.025571,10.216162


In [30]:
# #Gt member player codes from the latest GW
# query = """
#         WITH
#         currentgw AS (
#             SELECT MAX(gw) as max_gw
#             FROM playergw)
#         SELECT *
#         FROM playergw
#         WHERE gw = currentgw
#         ;
#         """
# with fpl_engine.connect() as conn:
#     viewdf = pd.read_sql_query(query, conn)
# print(viewdf.shape)

In [31]:
# #Query gets only player details from the latest GW
# query = """
#         ALTER TABLE player_details
#             RENAME COLUMN total_points TO total_points_gw;

#         WITH playergw AS (
#             SELECT *,
#             ROW_NUMBER() OVER(
#                     PARTITION BY
#                         season,
#                         code,
#                         event,
#                         fixture_id --Only one record per GW
#                     ORDER BY 
#                         now_datetime_utc DESC) as rn --Only select the latest date
#         FROM playergw
#         ),
#         currentgw AS (
#             SELECT MAX(current_gw) as max_gw
#             FROM playergw
#         )
        
#         SELECT *
#         FROM player_details
#         LEFT JOIN playergw
#             ON player_details.element = playergw.id
#             AND player_details.round = playergw.current_gw
#         CROSS JOIN currentgw
#         WHERE player_details.round = currentgw.max_gw
#             AND playergw.rn = 1
#         ;
#         """
# with engine.connect() as conn:
#     playerdf = pd.read_sql_query(query, conn)
# print(playerdf.shape)

# pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

In [32]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.01 minute(s)

Finished in 0.0 minute(s)
